In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pickle import load

In [0]:
input_data = load(open('/content/drive/My Drive/Colab Notebooks/Movies/all_movie_scripts', 'rb'))

In [4]:
type(input_data[0])

str

In [5]:
input_data

['bingo fucking hell fucking hell oh god keep your shit together if you wanna get em out of here i can pop these no problem so get to it security we have a break in get the i know who you are i know what you think if you do n t wanna get hurt keep your mouth shut and do n t move the chimps are infected they re highly contagious they ve been given an inhibitor infected with what to cure you must first understand infected with what rage what is he talking about get the cages open no no no listen you sick bastard we re taking your torture victims with us we ll get you out of here the animals are contagious the infection is in their blood and saliva one bite stop stop you ve no idea get it off help me get it off get it off what s the matter with her you have to kill her oh my god what s the matter with her we have to kill her now oh god hello hello hello hello hello hello hello father father are you ok oh should n t have done that should n t have done that shit over here over here come on 

In [0]:
import spacy
import re
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'net'])

In [0]:
def process_data(data):
    return [token.text.lower() for token in nlp(data) if token.text not in '!@#$%^&*()_+~`{}[]:";,./<>?\n\n\n\t\b-...."']

def form_sequences(data):
    seq_len = 50 + 1
    sequences = []
    for i in range(seq_len, len(input_data)):
        sequences.append(input_data[i-seq_len:i])
    return sequences

In [0]:
input_data = ' '.join(input_data)

In [9]:
len(input_data)

272119

In [0]:
input_data = process_data(input_data)

In [0]:
sequences = form_sequences(input_data)

In [12]:
len(sequences[0])

51

In [13]:
!pip install keras
!pip install tensorflow

In [14]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
tokenizer = Tokenizer()

In [0]:
tokenizer.fit_on_texts(sequences)

In [0]:
sequences = tokenizer.texts_to_sequences(sequences)

In [18]:
sequences

[[5128,
  102,
  144,
  102,
  144,
  46,
  87,
  107,
  38,
  63,
  428,
  61,
  1,
  335,
  19,
  230,
  35,
  11,
  26,
  3,
  44,
  1173,
  127,
  21,
  706,
  55,
  19,
  5,
  6,
  397,
  8,
  32,
  7,
  641,
  17,
  19,
  2,
  3,
  45,
  104,
  1,
  24,
  3,
  45,
  15,
  1,
  86,
  61,
  1,
  13,
  18],
 [102,
  144,
  102,
  144,
  46,
  87,
  107,
  38,
  63,
  428,
  61,
  1,
  335,
  19,
  230,
  35,
  11,
  26,
  3,
  44,
  1173,
  127,
  21,
  706,
  55,
  19,
  5,
  6,
  397,
  8,
  32,
  7,
  641,
  17,
  19,
  2,
  3,
  45,
  104,
  1,
  24,
  3,
  45,
  15,
  1,
  86,
  61,
  1,
  13,
  18,
  9],
 [144,
  102,
  144,
  46,
  87,
  107,
  38,
  63,
  428,
  61,
  1,
  335,
  19,
  230,
  35,
  11,
  26,
  3,
  44,
  1173,
  127,
  21,
  706,
  55,
  19,
  5,
  6,
  397,
  8,
  32,
  7,
  641,
  17,
  19,
  2,
  3,
  45,
  104,
  1,
  24,
  3,
  45,
  15,
  1,
  86,
  61,
  1,
  13,
  18,
  9,
  335],
 [102,
  144,
  46,
  87,
  107,
  38,
  63,
  428,
  61,
  1,
  335,


In [19]:
tokenizer.word_index

{'you': 1,
 'the': 2,
 'i': 3,
 's': 4,
 'to': 5,
 'it': 6,
 'a': 7,
 'we': 8,
 't': 9,
 'that': 10,
 'of': 11,
 'and': 12,
 'do': 13,
 'is': 14,
 'what': 15,
 'on': 16,
 'in': 17,
 'n': 18,
 'get': 19,
 'this': 20,
 'no': 21,
 'me': 22,
 're': 23,
 'are': 24,
 'there': 25,
 'here': 26,
 'they': 27,
 'all': 28,
 'come': 29,
 'm': 30,
 'go': 31,
 'have': 32,
 'he': 33,
 'be': 34,
 'out': 35,
 'not': 36,
 'got': 37,
 'your': 38,
 'for': 39,
 'my': 40,
 'up': 41,
 'll': 42,
 'just': 43,
 'can': 44,
 'know': 45,
 'oh': 46,
 'right': 47,
 'yeah': 48,
 'was': 49,
 'let': 50,
 'with': 51,
 'us': 52,
 'them': 53,
 'him': 54,
 'so': 55,
 'down': 56,
 'okay': 57,
 'one': 58,
 'now': 59,
 'but': 60,
 'if': 61,
 'back': 62,
 'shit': 63,
 'at': 64,
 'she': 65,
 've': 66,
 'like': 67,
 'how': 68,
 'hey': 69,
 'gon': 70,
 'na': 71,
 'did': 72,
 'see': 73,
 'her': 74,
 'about': 75,
 'going': 76,
 'want': 77,
 'fuck': 78,
 'where': 79,
 'look': 80,
 'off': 81,
 'need': 82,
 'from': 83,
 'were': 84,
 'm

In [0]:
vocab_size = len(tokenizer.word_counts)

In [0]:
import numpy as np

In [0]:
sequences = np.array(sequences)

In [23]:
sequences

array([[5128,  102,  144, ...,    1,   13,   18],
       [ 102,  144,  102, ...,   13,   18,    9],
       [ 144,  102,  144, ...,   18,    9,  335],
       ...,
       [ 557,   17, 2673, ...,  162,  129,  883],
       [  17, 2673, 1172, ...,  129,  883,  120],
       [2673, 1172,   16, ...,  883,  120,   43]])

In [0]:
X = sequences[:,:-1]
y = sequences[:, -1]

In [0]:
from keras.utils import to_categorical

In [26]:
X

array([[5128,  102,  144, ...,   61,    1,   13],
       [ 102,  144,  102, ...,    1,   13,   18],
       [ 144,  102,  144, ...,   13,   18,    9],
       ...,
       [ 557,   17, 2673, ...,   39,  162,  129],
       [  17, 2673, 1172, ...,  162,  129,  883],
       [2673, 1172,   16, ...,  129,  883,  120]])

In [0]:
y = to_categorical(y, num_classes=vocab_size+1)

In [28]:
y.shape

(56433, 5129)

In [0]:
seq_len = X.shape[1]

In [30]:
seq_len

50

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

In [0]:
def create_model(vocab_size, seq_len):
    
    model = Sequential()
    model.add(Embedding(vocab_size, seq_len, input_length=seq_len))
    model.add(LSTM(500, return_sequences=True))
    model.add(LSTM(400))
    model.add(Dense(350, activation='relu'))
    model.add(Dense(250, activation='relu'))
    
    model.add(Dense(vocab_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    
    model.summary()
    
    return model

In [33]:
model = create_model(vocab_size+1, seq_len)






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            256450    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 500)           1102000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 400)               1441600   
_________________________________________________________________
dense_1 (Dense)              (None, 350)               140350    
_________________________________________________________________
dense_2 (Dense)              (None, 250)               87750     
_________________________________________________________________
dense_3 (Dense)              (None, 5129)              1287379   
Total params: 4,315,529
Trainable params: 4,315,529
Non-trainable params: 0
_______________________________________

In [35]:
 model.fit(X, y, batch_size=200, epochs=200, verbose=1)

Epoch 1/200
56433/56433 [==============================] - 51s 897us/step - loss: 0.1466 - acc: 0.9566
Epoch 2/200
21000/56433 [==========>...................] - ETA: 31s - loss: 0.1649 - acc: 0.9507

KeyboardInterrupt: ignored

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [0]:
model.save('zombie_movie_96.h5')
model_file = drive.CreateFile({'title' : 'zombie_movie_96.h5'})    
model_file.SetContentFile('zombie_movie_96.h5')
model_file.Upload()

In [41]:
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1KUjagpZu-_PfRqANiRKo_m3YhTiSHg3Z'})

In [36]:
model

In [37]:
model.evaluate(X,y)

56433/56433 [==============================] - 135s 2ms/step


[0.1360203179060476, 0.9604486736505096]

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_word_index = model.predict_classes(pad_encoded)[0]
        pred_word = tokenizer.index_word[pred_word_index]
        
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
    
    return ' '.join(output_text)

In [45]:
sequences[267]

array([ 521,    5,  180,   12,    2, 2681,  486, 1174,    1,   96,   18,
          9,  180,   10, 1451,   25,   33,  486,   21,    6,    4,   36,
          7, 2682,    6,    4,    7, 1450,   33,    4,  645, 2683,    1,
        137,  124,   19,   16,   67,    7,  398,   16,  275,   55,  104,
         24,    1,    1,   23,   83,    2,  430])

In [0]:
seed_text = 'Who would have thought that an accident involving lime soda would set off the apocalypse? And when the contaminated green haze blew west. Because it turned them into... Zombies. That\'s right. Zombies!But we built a big barrier to protect us from the zombie hordes.'

In [52]:
len(seed_text)

264

In [54]:
generate_text(model, tokenizer, seq_len, seed_text, num_gen_words=1000)

'right you can still be walking round in this old world alone cause after life is over the afterlife goes you have that maverick the skybridge so what do we have me choice then it might be here um he was probably just fit never too happy but i do n t make it i talk to pay about you she no i m trying to find us maybe its things please stop i are you and your educated skull it me a favor a asses feedback almost made it out contact dumb tape what about it a cat representation of the man defense unstable sir uh muttering very fell the hive i see the rest of you and that is that i don t want to split you though hot cross the president so there s late for us if from the post that is the cell all has four continues blades whirring they are inbound to need off it would shoot try to come back about the blockade it makes you might have done to have that now the dj and you for this thing out of here this than dying i can i shoot about this do n t get it you go thank you all shit get back to save 